In [ ]:
pip install gradio torch jupyter ipywidgets transformers ollama

In [ ]:
import ollama
ollama.pull('mistral')

In [ ]:
from ollama import Client
client = Client(host='http://localhost:11434')
response = client.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
import ollama

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

In [ ]:
ollama.list()

In [ ]:
ollama.show('mistral:7b')

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

def mistralChat(text):
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16)
    model = model.to('cuda')
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    
    input_length = inputs.input_ids.shape[1]
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7, 
                            return_dict_in_generate=True,do_sample=True)
    
    tokens = outputs.sequences[0, input_length:]
    return tokenizer.decode(tokens)


In [62]:
with gr.Blocks() as server:
    with gr.Tab("LLM Inferencing"):
    
        model_input = gr.Textbox(label="Your Question:", 
                                value="What’s your question?", interactive=True)
        ask_button = gr.Button("Ask")
        model_output = gr.Textbox(label="The Answer:", interactive=False, 
                                value="Answer goes here...")
    
    ask_button.click(mistralChat, inputs=[model_input], outputs=[model_output])

server.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://f7bb4744bcff4e0717.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.38s/it]
Traceback (most recent call last):
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/blocks.py", line 1741, in process_api
    result = await self.call_function(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/blocks.py", line 1296, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/home/kaminaduc